# Asistente Turístico de Tenerife
**Práctica Final - Large Language Models**

In [1]:
# Section 1: Setup and PDF Loading
from src.conf import OPENAI_API_KEY, MODEL_CONFIG, PDF_PATH
from src.logger import logger
from src.api_client import OpenAIClient
from src.data_loader import DataLoader

print(f"API Key loaded: {OPENAI_API_KEY[:8]}...")
print(f"Model config: {MODEL_CONFIG}")

API Key loaded: sk-proj-...
Model config: {'model': 'gpt-4o-mini', 'temperature': 0.3, 'max_tokens': 1024, 'top_p': 0.9}


In [2]:
# Initialize OpenAI client
client = OpenAIClient()

if client.test_connection():
    print("✅ OpenAI connection OK")
else:
    print("❌ OpenAI connection failed")

2026-01-14 16:35:08,996 - INFO - OpenAI client initialized with model: gpt-4o-mini
2026-01-14 16:35:10,526 - INFO - OpenAI connection test successful


✅ OpenAI connection OK


In [3]:
# Load PDF
loader = DataLoader(PDF_PATH)
pages = loader.load()
stats = loader.get_stats()
print(f"✅ PDF loaded: {stats}")

2026-01-14 16:35:11,294 - INFO - DataLoader initialized with: data/TENERIFE.pdf
2026-01-14 16:35:11,638 - INFO - Loaded 25 pages from data/TENERIFE.pdf


✅ PDF loaded: {'num_pages': 25, 'total_chars': 16094, 'total_words': 2817, 'avg_chars_per_page': 643}


In [4]:
# Preview first page content
print(loader.get_page(0)[:500])

TENERIFE – LUGARES DE INTERÉS 
SITIOS QUE VER 
 
ZONA NORTE 
 
• Santa Cruz de Tenerife: 
Santa Cruz de Tenerife es la capital de la isla. Quizás la ruta a seguir si vais a Santa 
Cruz sería: 
- Aparcar en el aparcamiento del Parque Marítimo (ubicación). 
- Caminar por la Avenida Marítima hasta Plaza de España (ubicación). 
- Por el camino de la Avenida Marítima, ver el auditorio de Tenerife (ubicación). 
- Una vez llegados a Plaza España, callejear un poco (subir la Calle Castillo 
dirección Pl


In [5]:
# Section 2: Vector Store (RAG)
from src.vector_store import VectorStore

# Build vector store from loaded pages
vector_store = VectorStore()
num_chunks = vector_store.build_from_documents(pages)
print(f"✅ Vector store built: {num_chunks} chunks")
print(f"Chunk stats: {vector_store.get_chunk_stats()}")

2026-01-14 16:35:19,908 - INFO - VectorStore initialized (chunk_size=1000, overlap=200)
2026-01-14 16:35:19,909 - INFO - Created 30 chunks from 25 pages
2026-01-14 16:35:21,037 - INFO - Vector store built and persisted to ./chroma_db


✅ Vector store built: 30 chunks
Chunk stats: {'num_chunks': 30, 'avg_chunk_size': 570, 'min_chunk_size': 86, 'max_chunk_size': 987}


In [6]:
# Test retrieval - search for relevant chunks
query = "¿Cómo puedo subir al Teide?"
results = vector_store.search(query, k=3)

print(f"Query: {query}\n")
for i, doc in enumerate(results):
    print(f"--- Result {i+1} (page {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content[:300])
    print()

2026-01-14 16:35:28,331 - INFO - Search query: '¿Cómo puedo subir al Teide?...' returned 3 results


Query: ¿Cómo puedo subir al Teide?

--- Result 1 (page 14) ---
• El Teide 
Si vais a Tenerife y no subís al Parque Nacional del Teide, simplemente no habéis 
estado en Tenerife. 
 
Mi recomendación – esto es simplemente gustos personales míos – para subir al 
Teide es subir por la carretera TF24 (carretera de La Esperanza) desde la rotonda de 
Padre Anchieta en

--- Result 2 (page 15) ---
Si queréis subir hasta el pico del Teide, podéis hacerlo desde aquí haciendo uso de 
los teleféricos del Teide. Y, si queréis más info sobre el Teide, podéis ir al Centro de 
Visitantes de El Portillo que es gratis. 
Si os apetece, podéis subir de noche cuando esté despejado, ya que desde El Teide 


--- Result 3 (page 13) ---
o Si vais a llegar hasta Punta de Teno, consultad los horarios y los accesos, ya 
que creo que actualmente solo se puede ir en guagua (línea 369 de Titsa) 
desde la Estación de Buenavista. 
 
Curiosidades: En la carretera de Punta de Teno se grabó el principio de la 
película de

In [7]:
# Test another query
query = "¿Dónde puedo comer en Tenerife?"
results = vector_store.search(query, k=3)

print(f"Query: {query}\n")
for i, doc in enumerate(results):
    print(f"--- Result {i+1} ---")
    print(doc.page_content[:300])
    print()

2026-01-14 16:35:48,595 - INFO - Search query: '¿Dónde puedo comer en Tenerife?...' returned 3 results


Query: ¿Dónde puedo comer en Tenerife?

--- Result 1 ---
piscinas naturales. 
Una vez llegados a la Playa Martiánez, volvería por el mismo camino hasta el 
principio de la calle de San Telmo y bajaría por la calle Quintana hasta la Plaza del 
Charco (los helados del Freddino no están mal por si os apetece) y, de ahí, de vuelta 
al coche. 
En el Puerto de 

--- Result 2 ---
▪ Arepera La Carajita (entre La Orotava y el Puerto de la Cruz 
[ubicación]. Este sitio tiene las mejores arepas que podréis comer en 
vuestra vida y son baratísimas. Recomendación: La Fogalera y Reina 
Pepiada. Y pedir de entrantes una degustación de cachapitas). 
Importante ir con hambre.  
▪ Atuv

--- Result 3 ---
Guachinche es un sitio como el que he indicado.  
 
▪ Pizzería Gioconda (zona de La Orotava [ubicación]) [Tripadvisor] 
▪ El Palestra (zona de La Orotava [ubicación]. Este es el típico sitio que 
ha estado toda la vida en La Orotava y que te pides a domicilio cuando 
tienes el antojo. Nosotros solo 

